## Problématique

#### Identifier si une image provenant d'un anime est officielle ou un fan art à l'aide d'une prédiction par catégorie. Les données proviennent des résultats du moteur de recherche duckduckgo.com et seront labelisées à l'aide des mots-clés de recherche.

In [ ]:
# Installation de fastai
!pip install -Uqq fastai

### Téléchargement des images officielles et faites par des fans

In [ ]:
# Installation de duckduckgo_search pour faire les recherche d'images
!pip install -Uqq duckduckgo_search

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

# fonction qui permet de chercher des images
def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
# recherche d'une image avec les mots clés : "anime", "official" et "art"
urls = search_images('anime official art', max_images=1)
urls[0]

In [ ]:
# On télécharge l'image et on l'affiche
from fastdownload import download_url
dest = 'anime-official.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
# On fait de même avec les mots-clés : "anime", "fan" et "art". On télécharge et on affiche l'image
download_url(search_images('anime fan art', max_images=2)[1], 'anime-fan-art.jpg', show_progress=False)
Image.open('anime-fan-art.jpg').to_thumb(256,256)

#### Les images ont l'air de correspondre à notre problématique, nous allons donc maintenant en télécharger une certaines quantité et les séparer dans deux dossiers différents appelés anime-official et anime-fan-art

In [ ]:
path = Path('official-or-fan-art')

In [ ]:
searches = 'anime official art','anime fan art'

from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(o))
    sleep(10)  # Pause between searches to avoid over-loading server
    resize_images(path/o, max_size=400, dest=path/o)

#### Maintenant qu'on possède nos données, on vérifie si elles ont bien été téléchargées, si non on les supprime

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

#### On prépare les données pour les entraîner

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

### On entraîne le modèle avec le réseau de neurones resnet18

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

### Utilisation du modèle

In [ ]:
is_official,_,probs = learn.predict(PILImage.create('anime-official.jpg'))
print(f"This is a: {is_official}.")
print(f"Probability it's an anime official image: {probs[1]:.4f}")

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
# retourne la categorie prédit, l'index de la categorie prédit et la probabilité de chaque categorie
learn_inf.predict(path/'anime-fan-art.jpg')

In [ ]:
learn_inf.dls.vocab

In [ ]:
from fastai.vision.widgets import *

In [ ]:
#hide_output

btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
#hide
# For the book, we can't actually click an upload button, so we fake it
btn_upload = SimpleNamespace(data = ['anime-fan-art.jpg'])

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
#hide_output
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
#hide_output
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
#hide_output
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
#hide_output
VBox([widgets.Label('Select your image!'), 
      btn_upload, btn_run, out_pl, lbl_pred])